In [8]:
import requests
import time
import pandas as pd
from sqlalchemy import create_engine

In [9]:
host = ""
port = ""
user = ""
password = ""
database= ""

In [10]:
response = requests.get(f'https://api.opensensemap.org/boxes/590e0b0a51d3460011c725c4?format=json')
response = response.json()
response

{'_id': '590e0b0a51d3460011c725c4',
 'createdAt': '2022-03-30T11:25:43.281Z',
 'updatedAt': '2024-05-14T08:49:20.993Z',
 'name': 'Hafencity - Am Standtorkai/Osakaallee',
 'exposure': 'outdoor',
 'model': 'luftdaten_sds011',
 'sensors': [{'title': 'PM10',
   'unit': 'µg/m³',
   'sensorType': 'SDS 011',
   'icon': 'osem-cloud',
   '_id': '590e0b0a51d3460011c725c5',
   'lastMeasurement': {'createdAt': '2024-05-14T08:49:20.985Z',
    'value': '3.93'}},
  {'title': 'PM2.5',
   'unit': 'µg/m³',
   'sensorType': 'SDS 011',
   'icon': 'osem-cloud',
   '_id': '590e0b0a51d3460011c725c6',
   'lastMeasurement': {'createdAt': '2024-05-14T08:49:20.985Z',
    'value': '1.70'}},
  {'_id': '592469ca51d34600119bf21f',
   'icon': 'osem-thermometer',
   'sensorType': 'DHT22',
   'unit': '°C',
   'title': 'Temperatur',
   'lastMeasurement': {'createdAt': '2024-05-14T08:49:20.985Z',
    'value': '38.20'}},
  {'_id': '592469ca51d34600119bf220',
   'icon': 'osem-humidity',
   'sensorType': 'DHT22',
   'unit':

In [11]:
boxes_dic = {
    "Mersch Luxembourg" : "5ae4a726223bd80019a367a6",
    "Ettlingen Deutschland" : "5dde9523ba7944001da4150e",
    "Stuttgart Deutschland" : "5c08379b1c28f9001a3f580c",
    "München Deutschland" : "5b4d11485dc1ec001b5452c7",
    "Berlin Deutschland" : "65e8d93acbf5700007f920ca",
    "Hamburg Deutschland" : "590e0b0a51d3460011c725c4",
    "Dortmund Deutschland" : "605f498077a88b001bba3dc0",
    "Jena Deutschland" : "64722d1c9be0580007f776d9"
}

In [43]:
# Erstelle ein leeres DataFrame mit Spaltennamen
df = pd.DataFrame(columns=["Ort","Land","Time", "Temperature", "Lat", "Lon", "Hoehe","Luftfeuchtigkeit", "PM10", "PM2.5", "senseID"])

In [44]:
new_data = []  # Eine leere Liste, um neue Daten zu sammeln
for box_id, box_value in boxes_dic.items():
    response = requests.get(f'https://api.opensensemap.org/boxes/{box_value}?format=json')
    box = response.json() 
    temperature_value = None
    lat = None
    lon = None
    hoehe = None
    pm10_value = None
    pm25_value = None
    rel_Luftfeuchte_value = None
    ort_Land = box_id.split()

    # Extrahiere Koordinaten
    current_location = box.get('currentLocation', {})
    if current_location:
        coordinates = current_location.get('coordinates')
        if coordinates:
            lon = coordinates[0]
            lat = coordinates[1]
            height = coordinates[2] if len(coordinates) >= 3 else None

    # Traverse durch die 'sensors' Liste
    for sensor in box.get('sensors', []):
    # Überprüfe, ob der Sensor für Temperatur ist
        if sensor.get('title') == 'Temperatur':
           # Zugriff auf den Temperaturwert
            temperature_value = sensor.get('lastMeasurement', {}).get('value')
        
        if sensor.get('title') == 'PM10':
           # Zugriff auf den Temperaturwert
            pm10_value = sensor.get('lastMeasurement', {}).get('value')

        if sensor.get('title') == 'PM2.5':
           # Zugriff auf den Temperaturwert
            pm25_value = sensor.get('lastMeasurement', {}).get('value')

        if sensor.get('title') == 'rel. Luftfeuchte':
           # Zugriff auf den Temperaturwert
            rel_Luftfeuchte_value = sensor.get('lastMeasurement', {}).get('value')
            
            break  # Verlasse die Schleife, sobald die Temperatur gefunden ist
        
    # Füge neue Daten zur Liste hinzu
    new_data.append({"Ort": ort_Land[0], "Land":ort_Land[1], "Temperature": temperature_value, "Time": time.strftime("%Y-%m-%d %H:%M:%S"), "Lat": lat,"Lon":lon,"Hoehe":height, "Luftfeuchtigkeit": rel_Luftfeuchte_value, "PM10":pm10_value, "PM2.5":pm25_value, "senseID": box_value})
    
    # Konvertiere die neue Datenliste in ein DataFrame
new_df = pd.DataFrame(new_data)
    
    # Hänge das neue DataFrame an das bestehende DataFrame an
df = pd.concat([df, new_df], ignore_index=True)
    
    # Drucke oder verwende das DataFrame
print(df)


         Ort         Land                 Time        Temperature        Lat  \
0     Mersch   Luxembourg  2024-05-14 11:10:38              23.20  49.754663   
1  Ettlingen  Deutschland  2024-05-14 11:10:38              24.62  48.926209   
2  Stuttgart  Deutschland  2024-05-14 11:10:39              38.90  48.779610   
3    München  Deutschland  2024-05-14 11:10:39              17.10  48.146546   
4     Berlin  Deutschland  2024-05-14 11:10:40              25.20  52.511270   
5    Hamburg  Deutschland  2024-05-14 11:10:40              39.40  53.544046   
6   Dortmund  Deutschland  2024-05-14 11:10:41  25.72222222222222  51.497350   
7       Jena  Deutschland  2024-05-14 11:10:41              23.39  50.934561   

         Lon  Hoehe Luftfeuchtigkeit   PM10 PM2.5                   senseID  
0   6.105682  250.0            86.30   2.20  1.60  5ae4a726223bd80019a367a6  
1   8.374409    4.0            51.21   4.82  2.00  5dde9523ba7944001da4150e  
2   9.213723  280.0            32.60   1.40  

C:\Users\mariu\AppData\Local\Temp\ipykernel_18008\422821256.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_df], ignore_index=True)


In [45]:
df

,Ort,Land,Time,Temperature,Lat,Lon,Hoehe,Luftfeuchtigkeit,PM10,PM2.5,senseID
0,Mersch,Luxembourg,2024-05-14 11:10:38,23.20,49.754663,6.105682,250.0,86.30,2.20,1.60,5ae4a726223bd80019a367a6
1,Ettlingen,Deutschland,2024-05-14 11:10:38,24.62,48.926209,8.374409,4.0,51.21,4.82,2.00,5dde9523ba7944001da4150e
2,Stuttgart,Deutschland,2024-05-14 11:10:39,38.90,48.779610,9.213723,280.0,32.60,1.40,1.17,5c08379b1c28f9001a3f580c
3,München,Deutschland,2024-05-14 11:10:39,17.10,48.146546,11.576033,20.0,67.70,10.77,3.38,5b4d11485dc1ec001b5452c7
4,Berlin,Deutschland,2024-05-14 11:10:40,25.20,52.511270,13.397356,45.0,31.30,None,None,65e8d93acbf5700007f920ca
5,Hamburg,Deutschland,2024-05-14 11:10:40,39.40,53.544046,9.997499,NaN,39.60,5.53,1.90,590e0b0a51d3460011c725c4
6,Dortmund,Deutschland,2024-05-14 11:10:41,25.72222222222222,51.497350,7.494606,NaN,None,None,None,605f498077a88b001bba3dc0
7,Jena,Deutschland,2024-05-14 11:10:41,23.39,50.934561,11.589701,146.0,39.82,5.40,1.98,64722d1c9be0580007f776d9
